In [5]:
import pandas as pd
import plotly.graph_objects as go

In [6]:
ma_test_res = pd.read_pickle('ma_test_res.pkl')
all_trades = pd.read_pickle('all_trades.pkl')

In [7]:
all_trades.shape

(64416, 11)

In [8]:
ma_test_res.shape

(630, 8)

In [80]:
ma_test_res.head(2)

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,562,-966.2,4,8,MA_4_8
1,GBP_USD,319,-684.2,4,16,MA_4_16


In [81]:
df_good = ma_test_res[(ma_test_res.CROSS.isin(crosses))&(ma_test_res.total_gain>0)].copy()

In [90]:
df_good.head()

,pair,num_trades,total_gain,mashort,malong,CROSS
4,GBP_USD,155,184.8,8,32,MA_8_32
24,GBP_USD,43,706.8,4,256,MA_4_256
34,GBP_CAD,143,1024.2,8,32,MA_8_32
54,GBP_CAD,53,379.0,4,256,MA_4_256
64,GBP_JPY,149,1129.0,8,32,MA_8_32


In [96]:
our_pairs = list(df_good.pair.value_counts()[:6].index)
our_pairs

['USD_JPY', 'GBP_JPY', 'AUD_JPY', 'AUD_CAD', 'NZD_USD', 'EUR_NZD']

In [98]:
all_trades['CROSS'] = "MA_" + all_trades.MASHORT.map(str) + "_" + all_trades.MALONG.map(str)

In [108]:
trades_aud_jpy = all_trades[(all_trades.CROSS=="MA_8_16")&(all_trades.PAIR=="AUD_JPY")].copy()
trades_aud_jpy["CUM_GAIN"] = trades_aud_jpy.GAIN.cumsum()
# trades_aud_jpy.head(5)
trades_aud_jpy.tail(5)

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS,CUM_GAIN
3901,2022-05-13 01:00:00+00:00,88.628,0.066688,-0.026938,1,25.5,25.5,AUD_JPY,8,16,75.0,MA_8_16,1685.5
3928,2022-05-16 04:00:00+00:00,88.883,-0.023062,0.080500,-1,55.9,-55.9,AUD_JPY,8,16,9.0,MA_8_16,1629.6
3937,2022-05-16 13:00:00+00:00,89.442,0.020750,-0.066750,1,139.7,139.7,AUD_JPY,8,16,30.0,MA_8_16,1769.3
3967,2022-05-17 19:00:00+00:00,90.839,-0.001125,0.032688,-1,8.5,-8.5,AUD_JPY,8,16,5.0,MA_8_16,1760.8
3972,2022-05-18 00:00:00+00:00,90.924,0.007625,-0.016625,1,-50.2,-50.2,AUD_JPY,8,16,3.0,MA_8_16,1710.6


In [112]:
def plot_line(df_plot, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot.CUM_GAIN,
        line=dict(width=2,color="#3d825f"),
        line_shape="spline",
        name=name,
        mode="lines"
    ))
    fig.update_layout(width=1000,height=400,
    margin=dict(l=15,r=15,b=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e",
    title=name)
    fig.update_xaxes(
        linewidth=1,
        linecolor="#3a4a54",
        showgrid=False,
        zeroline=False
    )
    fig.update_yaxes(
        linewidth=1,
        linecolor="#3a4a54",
        showgrid=False,
        zeroline=False
    )
    fig.show()

In [113]:
plot_line(trades_aud_jpy, "AUD_JPY")

In [99]:
all_trades.head(2)

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS
13,2021-09-28 21:00:00+00:00,1.35380,0.000242,-0.000152,1,-19.8,-19.8,GBP_USD,4,8,10.0,MA_4_8
23,2021-09-29 07:00:00+00:00,1.35182,-0.000111,0.000291,-1,-91.2,91.2,GBP_USD,4,8,16.0,MA_4_8


In [95]:
ma_test_res.columns

Index(['pair', 'num_trades', 'total_gain', 'mashort', 'malong', 'CROSS'], dtype='object')

In [11]:
ma_test_res = ma_test_res[['pair', 'num_trades', 'total_gain','mashort', 'malong']]

In [14]:
ma_test_res["CROSS"] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str)

In [27]:
df_all_gains = ma_test_res.groupby(by=["CROSS", "mashort", "malong"], as_index=False).sum()

In [28]:
df_all_gains.sort_values(by="total_gain", ascending=False, inplace=True)

In [52]:
df_all_gains.head()

,CROSS,mashort,malong,num_trades,total_gain
27,MA_8_32,8,32,3084,13801.7
2,MA_16_32,16,32,2738,12828.8
16,MA_4_256,4,256,1086,6692.5
26,MA_8_256,8,256,790,6544.5
6,MA_24_256,24,256,481,6222.8


In [40]:
ma_8_16 = ma_test_res[ma_test_res.CROSS=='MA_8_16'].copy()

In [47]:
ma_8_16.sort_values(by="total_gain", ascending=False, inplace=True)

In [55]:
total_p = len(ma_8_16.pair.unique())

In [57]:
ma_8_16[ma_8_16.total_gain>0].shape[0]

13

In [58]:
13/total_p

0.6190476190476191

In [78]:
for cross in df_all_gains.CROSS.unique():
    df_temp = ma_test_res[ma_test_res.CROSS==cross]
    
    total_p = df_temp.shape[0]
    n_good = df_temp[df_temp.total_gain>0].shape[0]
    print(f"cross: {cross:12}     n_good:{n_good:1}         percent:{(n_good/total_p)*100:4.0f}% ")

cross: MA_8_32          n_good:18         percent:  86% 
cross: MA_16_32         n_good:13         percent:  62% 
cross: MA_4_256         n_good:13         percent:  62% 
cross: MA_8_256         n_good:12         percent:  57% 
cross: MA_24_256        n_good:13         percent:  62% 
cross: MA_24_64         n_good:16         percent:  76% 
cross: MA_64_256        n_good:11         percent:  52% 
cross: MA_16_256        n_good:12         percent:  57% 
cross: MA_4_32          n_good:12         percent:  57% 
cross: MA_32_256        n_good:11         percent:  52% 
cross: MA_24_32         n_good:12         percent:  57% 
cross: MA_16_64         n_good:14         percent:  67% 
cross: MA_8_64          n_good:12         percent:  57% 
cross: MA_32_64         n_good:15         percent:  71% 
cross: MA_8_16          n_good:13         percent:  62% 
cross: MA_4_64          n_good:9         percent:  43% 
cross: MA_16_128        n_good:13         percent:  62% 
cross: MA_24_96         n_good:1

In [79]:
crosses = df_all_gains.CROSS.unique()[:3]
crosses

array(['MA_8_32', 'MA_16_32', 'MA_4_256'], dtype=object)